<a href="https://colab.research.google.com/github/abh22/chatBot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.8 MB/s eta 0:00:00


In [2]:
!pip install -q accelerate bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.7 MB/s eta 0:00:00


In [3]:
!pip install langchain pypdf faiss-cpu google-search-results sentence-transformers langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.6 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=63ad26b2a89ddda235d5c9f47385d

In [12]:
from transformers import AutoTokenizer , AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
import torch


In [5]:
model_name="mistralai/Mistral-7B-Instruct-v0.2"
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [6]:
# @title Quantization config
compute_dtype=getattr(torch,"float16")
bnb_config= BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

parameters of loading data, quantization type, data type for computations and wether to use double quantization or not.

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [8]:
llm=pipeline(
    model=model,
    task="text-generation",
    tokenizer=tokenizer,
    temperature=0.3,
    return_full_text=True,
    do_sample=True,
    max_new_tokens=1000,
)
#sampling: the model randomly selects the next token based on the probabilities assigned by the model.

In [13]:
mistral_llm = HuggingFacePipeline(pipeline=llm)

In [31]:
from google.colab import files
document=files.upload()

Saving 2307.06435.pdf to 2307.06435 (2).pdf


In [32]:
doc = PyPDFLoader("/content/"+list(document.keys())[0])

In [33]:
doc=doc.load()

In [34]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)

In [35]:
text=text_splitter.split_documents(doc)

In [36]:
db = FAISS.from_documents(text,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [37]:
retriever = db.as_retriever()

In [38]:
template="""[INST] You are helpful Question Answering Ai system, tha try to answer user's questions depending on the given context .
In case you don't have the answer just say I don't have any informations.
You need to reply only for question so if the user's text doesn't seems like question type don't give him any informations .
Here the context you need :
{context}
##
Question : {text}
[/INST]"""

prompt=PromptTemplate(
    input_variables=["text"],
    template=template
)
chain=LLMChain(prompt=prompt,llm=mistral_llm)
rag_chain = (
 {"context": retriever, "text": RunnablePassthrough()}
    | chain
)

In [39]:
result=rag_chain.invoke("What is this document for?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [40]:
print(result['text'])

[INST] You are helpful Question Answering Ai system, tha try to answer user's questions depending on the given context .
In case you don't have the answer just say I don't have any informations.
You need to reply only for question so if the user's text doesn't seems like question type don't give him any informations .
Here the context you need :
[Document(page_content='tion of general purpose, financial, general purpose instructions,', metadata={'source': '/content/2307.06435 (2).pdf', 'page': 9}), Document(page_content='questions regarding the usefulness of the retrieved documents.', metadata={'source': '/content/2307.06435 (2).pdf', 'page': 12}), Document(page_content='the meaning of their prompts?” arXiv preprint arXiv:2109.01247 , 2021.\n31', metadata={'source': '/content/2307.06435 (2).pdf', 'page': 42}), Document(page_content='from Project Gutenberg. It’s specifically designed to facilitate', metadata={'source': '/content/2307.06435 (2).pdf', 'page': 22})]
##
Question : What is t

In [41]:
result=rag_chain.invoke("Can you give me a summary of this document?")
dash_line='#'.join(''for i in range(100))
print(dash_line)
print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###################################################################################################
[INST] You are helpful Question Answering Ai system, tha try to answer user's questions depending on the given context .
In case you don't have the answer just say I don't have any informations.
You need to reply only for question so if the user's text doesn't seems like question type don't give him any informations .
Here the context you need :
[Document(page_content='2.5 PG19 [308]: This is a digital library of diverse books', metadata={'source': '/content/2307.06435 (2).pdf', 'page': 22}), Document(page_content='A. Chen, A. Goldie, A. Mirhoseini, C. McKinnon et al. , “Constitutional', metadata={'source': '/content/2307.06435 (2).pdf', 'page': 35}), Document(page_content='interested in details to the original works.\nA. Tokenization', metadata={'source': '/content/2307.06435 (2).pdf', 'page': 3}), Document(page_content='to draw insights from extensive informative summaries of the', met